## 1) Perceptron with 1 neuron for AND gate using step activation function
- Here we are manually set up the weights and bias.
- Here we are not training the model, because wts and bias are already predefined

In [ ]:
import tensorflow as tf
import numpy as np
import random
import os

# Reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed()

# Define AND gate inputs and outputs
X = np.array([[0, 0], # out -> 0
              [0, 1], # out -> 0
              [1, 0], # out -> 0
              [1, 1]  # out -> 1
              ], dtype=np.float32)

# We do not need y because we are not trianing

# Custom step activation function
def step_function(x):
    return tf.cast(tf.greater_equal(x, 0), tf.float32)

# Build model with 1 neuron and custom activation
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation=step_function, input_shape=(2,))
])

# Use Mean Squared Error since output is binary and step is non-differentiable
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
              loss='mse')

# Training may not work properly due to non-differentiable step function,
# so we'll manually set weights to make AND work
# wt for input1 = 1.0
# wt for input2 = 1.0
# bias = -1.5
model.layers[0].set_weights([np.array([[1.0], [1.0]]), np.array([-1.5])])

predictions = model.predict(X) # Predict

# Output
print("\nAND Gate with Step Function:")
for i, pred in enumerate(predictions):
    print(f"Input: {X[i]} → Output: {pred[0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step

AND Gate with Step Function:
Input: [0. 0.] → Output: 0.0
Input: [0. 1.] → Output: 0.0
Input: [1. 0.] → Output: 0.0
Input: [1. 1.] → Output: 1.0


## 2) Now lets create a perceptron with 1 neuron with step function that can simulate output
 ```
Input  -> Output
[0. 0.] → 0
[0. 1.] → 1
[1. 0.] → 0
[1. 1.] → 1
```
- Here we manually set the weights and bias
- Here we are not training the model, because wts and bias are already predefined

In [ ]:
import tensorflow as tf
import numpy as np
import random
import os

# Reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed()

# Define AND gate inputs and outputs
X = np.array([[0, 0], # out -> 0
              [0, 1], # out -> 1
              [1, 0], # out -> 0
              [1, 1]  # out -> 1
              ], dtype=np.float32)

# We do not need y because we are not trianing

# Custom step activation function
def step_function(x):
    return tf.cast(tf.greater_equal(x, 0), tf.float32)

# Build model with 1 neuron and custom activation
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation=step_function, input_shape=(2,))
])

# Use Mean Squared Error since output is binary and step is non-differentiable
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
              loss='mse')

# Training may not work properly due to non-differentiable step function,
# so we'll manually set weights to make AND work
# wt for input1 = 0.2
# wt for input2 = 0.5
# bias = -0.5
model.layers[0].set_weights([np.array([[0.2], [0.5]]), np.array([-0.5])])

predictions = model.predict(X) # Predict

# Output
print("\n Step Function:")
for i, pred in enumerate(predictions):
    print(f"Input: {X[i]} → Output: {pred[0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step

 Step Function:
Input: [0. 0.] → Output: 0.0
Input: [0. 1.] → Output: 1.0
Input: [1. 0.] → Output: 0.0
Input: [1. 1.] → Output: 1.0


## 3) Lets make above little more realistic
- Here we manually set the weights and bias
- Here we are not training the model, because wts and bias are already predefined

We'll treat:

x1 = temperature (in °C)

x2 = pressure (in kPa)

We want the neuron to fire (output = 1) only if:

temperature > 50, and

pressure > 100

Let's derive the weights and bias so that the neuron fires (output = 1) only when both:

temperature > 50

pressure > 100

This is an AND-like condition over two continuous variables.

A perceptron with a step activation computes:
- output = 1 if w1 * temp + w2 * pressure + b >= 0
- output = 0 , otherwise

i.e. we want following output using step function:
- For values just above (50, 100) → output = 1
- For values at or below (50, 100) → output = 0

###Strategy:###
Choose simple weights: w1 = 1, w2 = 1.

Then solve: temp + pressure + b >= 0

We want the neuron to fire at temp = 51 and pressure = 101, but not at or below 50, 100.

So:
51+101+b≥0⇒152+b≥0⇒b≥−152

and

50+100+b<0⇒150+b<0⇒b<−150

Thus, pick:

w1=1, w2=1, b=−151
​


In [ ]:
import tensorflow as tf
import numpy as np

# Sample realistic inputs
# Columns: [temperature, pressure]
# Inputs
X = np.array([
    [50, 100],   # exactly at threshold → 0
    [53, 105],   # both just above → 1
    [60, 120],   # both well above → 1
    [45, 90],    # both below → 0
    [40, 90],     # both below → 0

], dtype=np.float32)

# We do not need y because we are not training

# Custom step activation function
def step_function(x):
    return tf.cast(tf.greater_equal(x, 0), tf.float32)

# Define 1-neuron model with step activation
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation=step_function, input_shape=(2,))
])

# Manually set weights and bias to match threshold logic:
weights = np.array([[1.0], [1.0]])  # temp & pressure weights
bias = np.array([-151.0])           # threshold offset
model.layers[0].set_weights([weights, bias])


# Predict
predictions = model.predict(X)

# Output
print("Realistic Inputs - AND-like Step Function Output:")
for i, pred in enumerate(predictions):
    print(f"Input: {X[i]} → Output: {int(pred[0])}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Realistic Inputs - AND-like Step Function Output:
Input: [ 50. 100.] → Output: 0
Input: [ 53. 105.] → Output: 1
Input: [ 60. 120.] → Output: 1
Input: [45. 90.] → Output: 0
Input: [40. 90.] → Output: 0


## But this is not what we want. We want to train our model from X and y , and figure out weights and bias from the training data.

Switching to sigmoid allows your single-neuron perceptron to:
- Be trained automatically using a loss function and backpropagation
- Produce probabilistic outputs
- Generalize better on real-valued input spaces

## Lets Train our 1 neuron perceptron
### 4) 1 Neuron perceptron for AND gate using sigmoid activation function

In [ ]:
# 2000 epoch took 1 minute
import tensorflow as tf
import numpy as np
import random
import os

# For consistent results
def set_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed()

# AND gate dataset
X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]], dtype=np.float32)

y = np.array([[0],
              [0],
              [0],
              [1]], dtype=np.float32)

# Define 1-neuron model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(2,))
])

# Compile with better optimizer
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=2000, verbose=0)

# Predict
predictions = model.predict(X)

# Output
print("\nAND Gate Predictions:")
for i, pred in enumerate(predictions):
    print(f"Input: {X[i]} → Predicted: {pred[0]:.4f}, Rounded: {round(pred[0])}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step

AND Gate Predictions:
Input: [0. 0.] → Predicted: 0.0000, Rounded: 0
Input: [0. 1.] → Predicted: 0.0201, Rounded: 0
Input: [1. 0.] → Predicted: 0.0201, Rounded: 0
Input: [1. 1.] → Predicted: 0.9718, Rounded: 1


In [ ]:
# Get weights and biases from the Dense layer
weights, biases = model.layers[0].get_weights()

print("\nWeights:")
print(weights)

print("\nBias:")
print(biases)


Weights:
[[7.423303 ]
 [7.4233027]]

Bias:
[-11.307543]
